In [3]:
from keras.layers import Dense, Conv2D,BatchNormalization,MaxPooling2D,Flatten,Dropout
from keras import layers , regularizers,activations
from keras.datasets import mnist
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import os 

In [4]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus :
    tf.config.experimental.set_memory_growth(gpu,True)

In [6]:
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("No GPU detected")


GPU is available


In [7]:


train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_images = os.getcwd() + "/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values


def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [8]:
inputs = keras.Input(shape=(64,64,1))

x=Conv2D(
    filters=32,
    kernel_size=3,
    padding='same',
    kernel_regularizer= regularizers.l2(WEIGHT_DECAY),
    )(inputs)
x=BatchNormalization()(x)
x=activations.relu(x)

x=Conv2D(64,3,kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
x=BatchNormalization()(x)
x=activations.relu(x)
x=MaxPooling2D()(x)


x=Conv2D(64,3,kernel_regularizer=regularizers.l2(WEIGHT_DECAY),activation='relu')(x)

x=Conv2D(128,3,kernel_regularizer=regularizers.l2(WEIGHT_DECAY),activation='relu')(x)
x=MaxPooling2D()(x)

x=Flatten()(x)

x=Dense(128,activation='relu')(x)
x=Dropout(0.5)(x)
x=Dense(64,activation='relu')(x)

output1 = Dense(10,name='first_num',activation='softmax')(x)
output2=Dense(10,name='second_num',activation='softmax')(x)

In [9]:
model = keras.Model(inputs=inputs,outputs=[output1,output2])
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=[keras.losses.SparseCategoricalCrossentropy(),
          keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy']
)
model.fit(train_dataset,epochs=5)
model.evaluate(test_dataset)

Epoch 1/5
1000/1000 [==============================] - 118s 110ms/step - loss: 1.8556 - first_num_loss: 0.8554 - second_num_loss: 0.8704 - first_num_accuracy: 0.6967 - second_num_accuracy: 0.6933
Epoch 2/5
1000/1000 [==============================] - 45s 45ms/step - loss: 0.5572 - first_num_loss: 0.2300 - second_num_loss: 0.2320 - first_num_accuracy: 0.9232 - second_num_accuracy: 0.9232
Epoch 3/5
1000/1000 [==============================] - 44s 44ms/step - loss: 0.3969 - first_num_loss: 0.1586 - second_num_loss: 0.1506 - first_num_accuracy: 0.9476 - second_num_accuracy: 0.9504
Epoch 4/5
1000/1000 [==============================] - 44s 44ms/step - loss: 0.3235 - first_num_loss: 0.1219 - second_num_loss: 0.1189 - first_num_accuracy: 0.9610 - second_num_accuracy: 0.9613
Epoch 5/5
313/313 [==============================] - 14s 44ms/step - loss: 1.2546 - first_num_loss: 0.3215 - second_num_loss: 0.8550 - first_num_accuracy: 0.9064 - second_num_accuracy: 0.7774


[1.2546087503433228,
 0.32152503728866577,
 0.8549602627754211,
 0.9063500165939331,
 0.7773500084877014]